In [1]:
from Check_Chromedriver import Check_Chromedriver
from sys import stdout, exc_info
from traceback import print_exception

from selenium.webdriver.chrome.options import Options
from selenium import webdriver

import csv

TIMEOUT = 30  # Seconds.
PAGE_URL = "https://www.basketball-reference.com/wnba/years/2020-schedule.html"

saved_URLs = []


def main():
    """ Function main.  Watch for new online jobs on Wyzant.com.

    Parameters:
    Returns:
    """

    try:
        # Check the version of chromedriver.exe, and update when needed.
        Check_Chromedriver.driver_mother_path = "./"
        Check_Chromedriver.main()

        # The usual options.
        options = Options()
        options.headless = True
        options.add_argument("--window-size=1920,1080")

        # Create object for accessing HTML.
        driver = webdriver.Chrome(options=options)

        # Get HTML for first page.
        driver.get(PAGE_URL)

        # Get all HTML tables on current web page.
        tables = driver.find_elements_by_xpath("//table")
        if len(tables) == 0:
            "No tables to look at."
        else:
            # Iterate over all HTML tables in this web page.
            for table_obj in tables:
                # Get HTML table ID, needed to identify correct table to print.
                table_id = table_obj.get_attribute("id")
                # Get HTML table class, in case it's needed for troubleshooting.
                table_class = table_obj.get_attribute("class")
                # print(f"\nTABLE ID: '{table_id}', TABLE CLASS: '{table_class}'")

                # Get all rows in this HTML table.
                rows = table_obj.find_elements_by_xpath(".//tr")
                if len(rows) == 0:
                    "No rows to look at."
                elif table_id == "schedule":
                    # Only want to see table "schedule".
                    # Iterate over rows in this HTML table.
                    for row_obj in rows:
                        # The text of the row to print.
                        row_text = list()
                        # For troubleshooting.
                        # row_text.append("ROW:")

                        # Get cells in current row.
                        cells = row_obj.find_elements_by_xpath(".//th|.//td")
                        # Iterate over cells in current row.
                        for cell_number, cell_obj in enumerate(cells):
                            # Start assembling cell text.
                            cell_text = cell_obj.text
                            if cell_number == 1:
                                # This column holds the name of the visitor team.
                                visitor_team = cell_text
                            elif cell_number == 3:
                                # This column holds the name of the home team.
                                home_team = cell_text
                            elif cell_number == 5:
                                # This column holds the URL of the web page for how the teams did in this row's game.
                                anchors = cell_obj.find_elements_by_xpath(".//a")
                                anchors_list = []
                                for anchor in anchors:
                                    anchor_text = anchor.get_attribute("href")
                                    anchors_list.append(anchor_text)
                                if len(anchors) > 0:
                                    anchors_list = ",".join(anchors_list)
                                    cell_text += cell_text + " (" + anchors_list + ")"
                            # Assemble the text of the row to print.
                            row_text.append(cell_text)

                        # Print row.
                        print("\t".join(row_text))
                        if row_text[0] != "Date":
                            # Save the URL of the web page for how the teams did in this row's game.
                            # Also save the names of the two teams.
                            saved_URLs.append((anchors_list, visitor_team, home_team))
                            # print(anchors_list, visitor_team, home_team)

        # Now iterate over the list of web pages for the games.
        for item in saved_URLs:
            print()
            # Unpack "item"
            url = item[0]
            visitor_team = item[1]
            home_team = item[2]

            # Go to the next web page in the list.
            driver.get(url)

            # Get all HTML tables on current web page.
            tables = driver.find_elements_by_xpath("//table")
            if len(tables) == 0:
                "No tables to look at."
            else:
                # Keep track of the tables in this web page.
                count_tables = 0
                # Iterate over all HTML tables in this web page.
                for table_obj in tables:
                    # Get HTML table ID, needed to identify correct table to print.
                    table_id = table_obj.get_attribute("id")
                    # Get HTML table class, in case it's needed for troubleshooting.
                    table_class = table_obj.get_attribute("class")
                    # print(f"\nTABLE ID: '{table_id}', TABLE CLASS: '{table_class}'")

                    # Get the rows in this HTML table.
                    rows = table_obj.find_elements_by_xpath(".//tr")
                    if len(rows) == 0:
                        "No rows to look at."
                    elif table_id[0:10] == "box-score-":
                        # Print team role and name
                        count_tables += 1
                        if count_tables == 1:
                            print("VISITOR:", visitor_team)
                        elif count_tables == 2:
                            print("HOME:", home_team)

                        # Iterate over rows in this HTML table.
                        for row_obj in rows:
                            # The text of the row to print.
                            row_text = list()
                            # For troubleshooting.
                            # row_text.append("ROW:")

                            # Get cells in current row.
                            cells = row_obj.find_elements_by_xpath(".//th|.//td")
                            # Iterate over cells in current row.
                            for cell_number, cell_obj in enumerate(cells):
                                cell_text = cell_obj.text
                                row_text.append(cell_text)

                            # Print only the first and last rows.
                            if row_text[0] in {"Totals", "Player"}:
                                print("\t".join(row_text))
                                with open('parseTables2.csv', 'w') as file:
                                    csvwriter = csv.writer(file)
                                    csvwriter.writerows(row_text)

    
    except Exception:
        # Print exception.
        print_exception(*exc_info(), limit=None, file=stdout)
# End of function main.


if __name__ == '__main__':
    main()

chromebrowser_ver : 84.0.4147.105
chromedriver_ver : 84.0.4147.30
Date	Visitor/Neutral	PTS	Home/Neutral	PTS	 
Sat, Jul 25, 2020	Seattle Storm	87	New York Liberty	71	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007250NYL.html)
Sat, Jul 25, 2020	Los Angeles Sparks	99	Phoenix Mercury	76	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007250PHO.html)
Sat, Jul 25, 2020	Indiana Fever	76	Washington Mystics	101	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007250WAS.html)
Sun, Jul 26, 2020	Dallas Wings	95	Atlanta Dream	105	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007260ATL.html)
Sun, Jul 26, 2020	Chicago Sky	88	Las Vegas Aces	86	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007260LVA.html)
Sun, Jul 26, 2020	Connecticut Sun	69	Minnesota Lynx	77	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202007260MIN.html)
Tue, Jul 28, 2

Sat, Aug 15, 2020	Los Angeles Sparks	90	Indiana Fever	76	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008150IND.html)
Sat, Aug 15, 2020	Washington Mystics	73	Las Vegas Aces	88	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008150LVA.html)
Sat, Aug 15, 2020	New York Liberty	64	Minnesota Lynx	94	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008150MIN.html)
Sun, Aug 16, 2020	Chicago Sky	92	Atlanta Dream	67	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008160ATL.html)
Sun, Aug 16, 2020	Seattle Storm	95	Connecticut Sun	72	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008160CON.html)
Sun, Aug 16, 2020	Dallas Wings	95	Phoenix Mercury	89	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202008160PHO.html)
Tue, Aug 18, 2020	Las Vegas Aces	82	Chicago Sky	84	Box ScoreBox Score (https://www.basketball-reference.com/wnba/boxscores/202

Tue, Sep 8, 2020	Los Angeles Sparks		New York Liberty		 (https://www.basketball-reference.com/wnba/boxscores/202009080NYL.html)
Tue, Sep 8, 2020	Minnesota Lynx		Washington Mystics		 (https://www.basketball-reference.com/wnba/boxscores/202009080WAS.html)
Wed, Sep 9, 2020	Atlanta Dream		Chicago Sky		 (https://www.basketball-reference.com/wnba/boxscores/202009090CHI.html)
Wed, Sep 9, 2020	Phoenix Mercury		Connecticut Sun		 (https://www.basketball-reference.com/wnba/boxscores/202009090CON.html)
Wed, Sep 9, 2020	Dallas Wings		Seattle Storm		 (https://www.basketball-reference.com/wnba/boxscores/202009090SEA.html)
Thu, Sep 10, 2020	New York Liberty		Indiana Fever		 (https://www.basketball-reference.com/wnba/boxscores/202009100IND.html)
Thu, Sep 10, 2020	Washington Mystics		Los Angeles Sparks		 (https://www.basketball-reference.com/wnba/boxscores/202009100LAS.html)
Thu, Sep 10, 2020	Las Vegas Aces		Minnesota Lynx		 (https://www.basketball-reference.com/wnba/boxscores/202009100MIN.html)
Fri, Se